<a href="https://colab.research.google.com/github/secutron/Practice_Ignite/blob/main/2_1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##2.1.구글 Colab 기반 

이그나이트는 작성 시점을 기준으로 다음의 백엔드들을 지원한다.

- backends from native torch distributed configuration: “nccl”, “gloo”, “mpi”
- XLA on TPUs via pytorch/xla
- using Horovod framework as a backend

각 의미를 이해하기 위해 가능한 단순한 내용의 코드를 준비하고, 이를 구글 Colab 기반으로 동작시켜 본다.


### 2.1.3. TPU단 분산처리

 구글 Colab은, 별도로 런타임 유형을 지정하지 않는 경우 GPU나 TPU가 없는 VM(Virtual Machine)이 기본 할당된다. TPU단에서 이그나이트를 사용해보기 위해서는, Colab 페이지의 상단메뉴에서 ‘런타임’ > ‘런타임 유형 변경’ 선택 시 나오는 대화상자에서, 하드웨어 가속기를 아래 그림에서처럼 TPU로 변경한다.

 만일 가이드라인 문서에서 제시된 링크([2_1_3.ipynb](https://github.com/secutron/Practice_Ignite/blob/main/2_1_3.ipynb))를 통해 본 colab 페이지에 접근한 경우라면, 이미 TPU 사용으로 지정되어 있는 상태일 것이다. ~~마지막으로 사용한 런타임 유형 상태가 저장된다~~
  

<div align="center">
<img width=512 src="https://i.imgur.com/gcm9MlH.png"/>
</div>



####2.1.3.1. xla 지원 런타임 설치

 GPU 사용 시에는 런타임 유형 변경에서 GPU를 선택하는 것만으로 GPU를 사용할 수 있었다. 하지만 TPU 경우에는, 런타임 유형 변경에서 TPU를 선택한 이후 필요한 추가 환경설정 작업을 진행해 주어야 한다. 다행히 파이토치 커뮤니티에서는 아래와 같이 사용이 편리한 스크립트를 제공하고 있다. 

 ** 가이드라인 작성 시점 기준 잔여 [issue](https://github.com/pytorch/builder/pull/750)로, 20210304 version으로 지정함


In [8]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

# 11SEP21 nightly 동작 확인
VERSION = "nightly"

#https://github.com/pytorch/builder/pull/750
#VERSION = "20210304" # was 20200607" 

!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION
!pip install --pre pytorch-ignite

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  31776      0 --:--:-- --:--:-- --:--:-- 31776
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-nightly ...
Found existing installation: torch 1.9.0a0+gitc4c77e2
Uninstalling torch-1.9.0a0+gitc4c77e2:
  Successfully uninstalled torch-1.9.0a0+gitc4c77e2
Found existing installation: torchvision 0.9.0a0+7d41547
Uninstalling torchvision-0.9.0a0+7d41547:
  Successfully uninstalled torchvision-0.9.0a0+7d41547
Copying gs://tpu-pytorch/wheels/torch-nightly-cp37-cp37m-linux_x86_64.whl...
| [1 files][145.0 MiB/145.0 MiB]                                                
Operation completed over 1 objects/145.0 MiB.                                    
Copying gs://tpu-pytorch/wheels/torch_xla-nightly-cp37-cp37m-linux_x86_64.whl...
| [1 files][137.8 MiB/137.8 MiB]                       

####2.1.3.2. 패키지 설치

 TPU 사용 가능한 런타임으로 변경 되었다면 다음과 같이 이그나이트의 최신 version을 설치한다. 현 시점 기준 Colab에서 제공하는 VM은, 이그나이트가 사전 설치되어 있지 않은 상태이기 때문이다.  ~~참고로 pip 명령어는 package installer for python)의 약자이며, 아래 명령문 실행 시 이그나이트의 pre-release를 PyPI(python package index)로부터 설치하게 된다.~~ 

In [9]:
!pip install --pre pytorch-ignite

결과에 따르면, 본 가이드라인 작성 시점의 최신 버전인 xxxx년 x월 xx일자 (예: 2021년 9월 10일자) 이그나이트 x.x.x (예: 0.5.0)가 설치 되었음을 알 수 있다..

#### 2.1.3.3. 노드 1개, 노드 당 프로세스 수 8개 실행

 TPU는 구글 독점 디바이스로, 외부에 판매하지 않는다. ~~(허접한??) Edge TPU는 논외.~~ 가이드라인 작성 시점 기준 TPU를 이용할 수 있는 방법은 무료인 Colab 또는 유료인 GCP를 이용하는 것이다. ~~아니면 구글에 취업하거나.~~ 

 모델 학습에 GPU를 사용할 때에는, 이용 가능한 GPU 수량에 따라 프로세스 수를 조절할 필요가 있다. ~~가난하면 Colab, 최소한 GPU 1장, 보통은 GPU 2장 이상, 럭셔리라면 GPU 8장 이상.~~ 반면에 Colab TPU를 이용한다면, 무조건 프로세스를 8개로 나누어 실행이 가능하다. 아래와 같이 총 8개의 코어(칩 4개, 칩당 코어 2개)를 담고 있는 TPU 보드 단위로 할당되기 때문이다

<div align="center">
<img width=200 src="https://i.imgur.com/p4zM87x.png"/>
</div>

 이를 시험해보기 위해 아래의 코드를 실행한다. 코드에서 수행하는 작업은 아래와 같다.

~~코드라인 27번부터 36번까지인 idist.Parallel 컨텍스트 매니징 방식만 주의해서 보자. 그 이외는 중요하지 않다~~

 (라인 1)에서는 이그나이트의 distributed 패키지를 idist라는 이름으로 불러들이고, (라인 31-33)에서는 백엔드로 xla를 할당하는 등의 설정 작업을 지정한 후, (라인 27)에서의 training 함수를, (라인 35-36)에서 컨택스트 매니징이 가능한 idist.Parallel을 이용하여 run 시킨다.

 (라인 27)의 training 함수는 몇 가지 정보를 출력한 후, 무의미한 작업을 반복하도록 작성되었다.

 그리고 (라인 32)에서는 분산처리 설정에 해당하는 dist_configs 딕셔너리에 nproc_per_node 키에 해당하는 값으로 8을 설정하였다. Colab 지원 TPU는 코어가 8개 탑재되어 있으므로, 8개의 자식 프로세스를 생성(spawn)하여 처리를 진행하라는 명령이다.



In [10]:
import ignite.distributed as idist

from functools import wraps
import time
import random

import torch

def fn_timer(function):
    @wraps(function)
    def function_timer(*args, **kwargs):
        t0 = time.time()
        result = function(*args, **kwargs)
        t1 = time.time()
        print (idist.get_rank(), " : Total time running %s: %s seconds" 
               % (function.__name__, str(t1-t0)))
        return result
    return function_timer

@fn_timer
def torch_random_sort(n):
    x = torch.randn(3,4)
    x = x.to(idist.device())
    for _ in range(n):
        sorted, indices = torch.sort(x)

def training(local_rank, config, **kwargs):
    print(idist.get_rank(), ': run with config:', config, '- backend=', idist.backend())
    torch_random_sort(250000) # was 2500000
  
backend = 'xla-tpu' # nccl #'gloo' # or "xla-tpu" or None
dist_configs = {'nproc_per_node': 8, "start_method": "fork"}  # or dist_configs = {...}
config = {'c': 12345}

with idist.Parallel(backend=backend, **dist_configs) as parallel:
    parallel.run(training, config, a=1, b=2)

2021-09-11 07:09:57,746 ignite.distributed.launcher.Parallel INFO: Initialized distributed launcher with backend: 'xla-tpu'
2021-09-11 07:09:57,747 ignite.distributed.launcher.Parallel INFO: - Parameters to spawn processes: 
	nproc_per_node: 8
	nnodes: 1
	node_rank: 0
	start_method: fork
2021-09-11 07:09:57,753 ignite.distributed.launcher.Parallel INFO: Spawn function '<function training at 0x7f19d7a004d0>' in 8 processes


0 : run with config: {'c': 12345} - backend= xla-tpu
6 : run with config: {'c': 12345} - backend= xla-tpu
5 : run with config: {'c': 12345} - backend= xla-tpu
4 : run with config: {'c': 12345} - backend= xla-tpu
3 : run with config: {'c': 12345} - backend= xla-tpu
1 : run with config: {'c': 12345} - backend= xla-tpu
2 : run with config: {'c': 12345} - backend= xla-tpu
7 : run with config: {'c': 12345} - backend= xla-tpu
6  : Total time running torch_random_sort: 6.230305433273315 seconds
5  : Total time running torch_random_sort: 6.263589143753052 seconds
2  : Total time running torch_random_sort: 6.233970880508423 seconds
4  : Total time running torch_random_sort: 6.26934289932251 seconds
7  : Total time running torch_random_sort: 6.283221244812012 seconds
0  : Total time running torch_random_sort: 6.311795234680176 seconds
1  : Total time running torch_random_sort: 6.316040515899658 seconds
3  : Total time running torch_random_sort: 6.319287538528442 seconds


2021-09-11 07:10:22,032 ignite.distributed.launcher.Parallel INFO: End of run


 결과의 첫번 째 출력 항목은 아래와 같으며, distributed launcher가 xla 백엔드로 초기화 되었음을 표시한다.

~~누군가의 실행에 따라 날짜 등은 계속 변한다. 그리고 이그나이트 버전 변경에 따라 표시되는 내용도 변할 수 있다.~~
  
>*2021-07-01 02:49:45,067 ignite.distributed.launcher.Parallel INFO: Initialized distributed launcher with backend: 'xla-tpu'*

 상기 ‘’’1.4 분산 딥러닝 기본 지식’’’ 항목에서 언급된 바와 같이 분산처리를 위해서는 컴퓨팅 코어 간 통신이 필요하며, 본 예제에서는 컴퓨팅 코어가 TPU인 경우이므로 xla가 백엔드로 이용된 것이다. 그리고 만일 이 컴퓨팅 코어가 CPU인 경우 gloo나 mpi 백엔드, GPU 경우에는 nccl 백엔드 이용이 가능하다. 

 이와 관련해서는 DISTRIBUTED COMMUNICATION PACKAGE - TORCH.DISTRIBUTED 페이지의 [rule of thumb](https://pytorch.org/docs/stable/distributed.html) 항목을 참조한다. ~~분산처리의 역사만큼이나 다양한 분산처리 방식이 존재한다. TCP 등을 이용해 직접 프로세스간 통신을 처리하는 방법도 있지만, 하이레벨 관점에서 CPU는 gloo, GPU는 nccl, TPU는 xla 백엔드를 사용해야 한다고 생각하는 것이 정신건강에 좋다.~~

 두번 째 출력 항목은 node 1개(nnodes: 1)에서, 노드 당 1개의 처리 작업 (nproc_per_node: 8)이 진행될 것임을 알려준다. 

>*2021-06-29 07:01:35,571 ignite.distributed.launcher.Parallel INFO: - Parameters to spawn processes:*  
>>*nproc_per_node: 8*  
	*nnodes: 1*  
	*node_rank: 0*  
	*start_method: fork*  

 세번 째 출력 항목은 각 프로세스 내에서 print문에 의한 출력과 측정된 소요 시간을 보여주고 있다. rank가 각 0에서 7번으로 표기되었고, 프로세스 총 합은 8개임을 보여 준다.

>*2021-07-01 02:49:45,074 ignite.distributed.launcher.Parallel INFO: Spawn function '<function training at 0x7f62a2a640e0>' in 8 processes*  
*7 : run with config: {'c': 12345} - backend= xla-tpu*  
*5 : run with config: {'c': 12345} - backend= xla-tpu*  
*1 : run with config: {'c': 12345} - backend= xla-tpu*  
*0 : run with config: {'c': 12345} - backend= xla-tpu*  
*2 : run with config: {'c': 12345} - backend= xla-tpu*  
*4 : run with config: {'c': 12345} - backend= xla-tpu*  
*6 : run with config: {'c': 12345} - backend= xla-tpu*  
*3 : run with config: {'c': 12345} - backend= xla-tpu*  
*7  : Total time running torch_random_sort: 5.765413999557495 seconds*  
*2  : Total time running torch_random_sort: 5.794928312301636 seconds*  
*5  : Total time running torch_random_sort: 5.8417181968688965 seconds*  
*6  : Total time running torch_random_sort: 5.851101398468018 seconds*  
*1  : Total time running torch_random_sort: 5.874312877655029 seconds*  
*3  : Total time running torch_random_sort: 5.838055372238159 seconds*  
*4  : Total time running torch_random_sort: 5.859879016876221 seconds*  
*0  : Total time running torch_random_sort: 5.914531946182251 seconds*  

 그리고 마지막 출력 항목은 분산처리가 모두 끝났음을 알리고 있다.

>*2021-07-01 02:50:09,216 ignite.distributed.launcher.Parallel INFO: End of run*


 여기서 프로세스 시작을 알리는 시간(세번 째 출력 항목)이 02시 49분 45초이고, 분산처리가 모두 끝났음을 알리는 시간(마지막 출력 항목)은 02시 50분 09초임을 확인해 보자. 약 5.7에서 5.9초 정도의 시간이 소요되는 작업(training 함수)을 총 8번 진행하였으나, 전체 처리 시간은 약 24초이다.
  
 ~~처리 시간은 Colab에서 할당해주는 VM 종류에 따라 변할 수 있다~~
 

#### 2.1.3.4. VM 리소스 확인

 이제 상기 분산처리 작업을 진행한 Colab의 VM에 대해 확인해보자.


In [11]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 b

 상기 실험 시 Colab으로부터 할당받은 VM은 총 2개의 인텔 제온 프로세서가 탑재되어 있고, 각 프로세서 당 코어 수는 1개이다. TPU를 이용해 작업하였으므로 CPU 정보는 중요하지 않다. 다만 ‘’’2.1.2. GPU 단 분산처리'’’ 시 할당받은 VM은 2.20GHz 였는데, 이번에는 ‘’’2.1.1. CPU 단 분산처리’’’와 동일하게 다시 2.30GHz를 할당받았다는 차이가 존재한다 
~~즉 VM 할당받을 때마다, 또는 런타임 변경 때마다 달라질 수 있다~~


>*processor	: 0*  
...  
*model name	: Intel(R) Xeon(R) CPU @ 2.30GHz*  
...  
*cpu cores	: 1*  
...  
*processor	: 1*  
...  
*model name	: Intel(R) Xeon(R) CPU @ 2.30GHz*  
...  
*cpu cores	: 1*  
...  


In [12]:
!pip install --upgrade tpubar
!pip install --upgrade git+https://github.com/trisongz/tpubar.git

from tpubar import TPUMonitor

monitor = TPUMonitor(tpu_name=os.environ.get('TPU_NAME', None), profiler='v2')
monitor.start()

with idist.Parallel(backend=backend, **dist_configs) as parallel:
    parallel.run(training, config, a=1, b=2)

  Cloning https://github.com/trisongz/tpubar.git to /tmp/pip-req-build-fzcxqu94
  Running command git clone -q https://github.com/trisongz/tpubar.git /tmp/pip-req-build-fzcxqu94


TPU v2-8 Matrix Units:            0.00% Utilization

TPU v2-8 Active Time:            0.00% Utilization

CPU Intel(R) Xeon(R)  @ 2.20GHz 1vCPU/2 Threads:            0.00% Utilization

RAM             0.00% Utilization

2021-09-11 07:10:44,466 ignite.distributed.launcher.Parallel INFO: Initialized distributed launcher with backend: 'xla-tpu'
2021-09-11 07:10:44,469 ignite.distributed.launcher.Parallel INFO: - Parameters to spawn processes: 
	nproc_per_node: 8
	nnodes: 1
	node_rank: 0
	start_method: fork
2021-09-11 07:10:44,471 ignite.distributed.launcher.Parallel INFO: Spawn function '<function training at 0x7f19d7a004d0>' in 8 processes


6 : run with config: {'c': 12345} - backend= xla-tpu
7 : run with config: {'c': 12345} - backend= xla-tpu
0 : run with config: {'c': 12345} - backend= xla-tpu
4 : run with config: {'c': 12345} - backend= xla-tpu
2 : run with config: {'c': 12345} - backend= xla-tpu
1 : run with config: {'c': 12345} - backend= xla-tpu
3 : run with config: {'c': 12345} - backend= xla-tpu
5 : run with config: {'c': 12345} - backend= xla-tpu
0  : Total time running torch_random_sort: 6.064126968383789 seconds
6  : Total time running torch_random_sort: 6.130752086639404 seconds
2  : Total time running torch_random_sort: 6.157059669494629 seconds
5  : Total time running torch_random_sort: 6.168396234512329 seconds
1  : Total time running torch_random_sort: 6.178622722625732 seconds
7  : Total time running torch_random_sort: 6.216586112976074 seconds
4  : Total time running torch_random_sort: 6.228344202041626 seconds
3  : Total time running torch_random_sort: 6.239790201187134 seconds


2021-09-11 07:12:07,861 ignite.distributed.launcher.Parallel INFO: End of run


 할당된 VM은 TPU v2-8을 포함하고 있음을 알 수 있다.

>...  
*PU v2-8 Matrix Units: ...*  
*TPU v2-8 Active Time: ...*  
*...*  

#### 2.1.3.5. 결과 해석

 이제까지 Colab 기반으로 TPU단에서 이그나이트를 이용해 작업을 처리하는 방식을 살펴보았다. 그리고 ‘’’2.1.3.4절의 VM 리소스 확인’’’ 항목에서, 분산처리를 위해 Colab으로부터 할당받은 VM에는 TPU V2-8을 1장 포함하고 있음을 확인하였으며, 이로부터 총 8개의 TPU 코어를 지원받을 수 있었다. 

 분산처리에서는 여러 개의 프로세스를 생성하여 동시에 작업을 진행시키는 것이 중요하다. 여기서 동시에 작업을 진행시킨다는 의미는 동시에 작업을 하는 것처럼 보이는 것([concurrent computing](https://en.wikipedia.org/wiki/Concurrent_computing))이 아니라, 말 그대로 동시에 작업이 진행되는 것([parallel computing](https://en.wikipedia.org/wiki/Parallel_computing))을 말한다. 

본 가이드라인에서 보인 바와 같이, Colab 기반으로 TPU를 이용하는 경우 프로세스를 8개까지 실행시킬 수 있다. 이는 Colab으로부터 할당받은 TPU 1장에 포함된 코어가 8개이기 때문이다. 

(ToDo: GCP Pod와의 차이점 추가 예정)

그리고 가이드라인 작성 시점에서 wandb(weight and bias) tool은 TPU utilization을 제공하지 않는다. 만일 상세한 profiling을 진행하고 싶다면, 아래의 별도 자료를 참고하도록 한다. 

 참고: [PyTorch/XLA TPU Profiling Colab tutorial](https://colab.research.google.com/github/pytorch/xla/blob/master/contrib/colab/pytorch-xla-profiling-colab.ipynb#scrollTo=iMdPRFXIn_jH)


Note: This is not an official [LG AI Research](https://www.lgresearch.ai/) product but sample code provided for an educational purpose

<br/>
author: John H. Kim
<br/>  
email: john.kim@lgresearch.ai / secutron@naver.com  
